# Analyze Angular Components and Generate a New Component

In this notebook, we will use the "Titan Text G1 - Premier" foundational model and the "Titan Embeddings G1 - Text" embeddings model to analyze the code structure of Angular components and generate a new component based on the analysis.

## Task 1: Environment Setup

First, we need to set up the environment by importing the necessary libraries and creating a service client.

In [ ]:
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime', region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Task 2: Load and Analyze Angular Components

Next, we will load the Angular components from the specified directories and use the embeddings model to analyze their structure.

In [ ]:
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import os

def load_text_files_recursive(directory):
    documents = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith((".ts", ".html", ".css", ".js")):
                filepath = os.path.join(root, file)
                print(f"Parsing file {filepath}")  # Added print statement
                try:
                    loader = TextLoader(filepath)
                    documents.extend(loader.load())
                except Exception as e:
                    print(f"Error loading file {filepath}: {e}")
    return documents

# Load Angular components recursively
documents = load_text_files_recursive("../src/app")
print(f"Loaded {len(documents)} documents.")

# Split documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
docs = text_splitter.split_documents(documents)
print(f"Split into {len(docs)} chunks.")

# Create embeddings
br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

# Create vector store
vectorstore_faiss = FAISS.from_documents(docs, embedding=br_embeddings)
print("Vector store created.")

## Task 3: Generate a New Component

Now, we will use the foundational model to generate a new Angular component based on the analysis stored in the vector database.

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock

# Initialize chat model
chat_model = ChatBedrock(model_id="amazon.titan-text-premier-v1:0", client=bedrock_client)

# Define system prompt
system_prompt = (
    "You are an assistant for generating Angular components. "
    "Use the following pieces of retrieved context to generate "
    "a new Angular component."
    "\n\n"
    "{context}"
)

# Create prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Generate an angular component with all the necessary ts, html, js and css files called lava-flow")
    ]
)

# Create retrieval chain
retriever = vectorstore_faiss.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Generate new component
response = rag_chain.invoke({"input": "Generate an angular component with all the necessary ts, html, js and css files called lava-flow"})

# Display the generated code in the specified format
generated_code = response['answer']
print(generated_code)

This concludes our experiment!